# Closed orbit calulator
based on the paper Nuclear Instruments and Methods in Physics Research A 647 https://doi.org/10.1016/j.nima.2011.05.052 Christian B "Cyclotron closed orbits on a radial grid"

In [1]:
#define working Dir 
TowerPC = True
Laptop = False
if(TowerPC):
    Dir          = "/mnt/c/Users/stu_w/OneDrive/codes/Opal/External_Input_Maps"
    OpalFunPath  = "/mnt/c/Users/stu_w/OneDrive/codes/Opal/JupiterNotes/Opal/My_Opal_functions.ipynb"
    GenToolsPath = "/mnt/c/Users/stu_w/OneDrive/codes/Opal/JupiterNotes/Utilities/GeneralTools.ipynb"
    
elif(Laptop):
    Dir          = "/mnt/c/Users/stu_w/OneDrive/codes/Opal/OPALFruit/tunes/External_Input_Maps"
    OpalFunPath  = "/mnt/c/Users/stu_w/OneDrive/codes/Opal/JupiterNotes/Opal/My_Opal_functions.ipynb"
    GenToolsPath = "/mnt/c/Users/stu_w/OneDrive/codes/Opal/JupiterNotes/Utilities/GeneralTools.ipynb"
    
else:
    Dir =  "/home/stu/OPALFruit/tunes/External_Input_Maps"
    OpalFunPath = '/home/stu/Software/JupiterNotes/Opal/My_Opal_functions.ipynb'
    GenToolsPath = '/home/stu/Software/JupiterNotes/Utilities/GeneralTools.ipynb'
   
%run {OpalFunPath}
%run {GenToolsPath}

## Units and simple functions 

In [2]:
import numpy as np
import scipy.constants as const 


# define  units to use
DefEunit = 'MeV'
DefLunit = 'mm'
DefBunit = 'kGauss'

EUnit = {'TeV':1e12 ,'GeV': 1e9,'MeV':1e6 ,'KeV':1e3 ,'eV': 1 ,'meV':1e-3 ,'ueV':1e-6 } 
LUnit = {'km':1e3 ,'m': 1 ,'mm':1e-3 ,'um':1e-6 } 
BUnit = {'kGauss':1e3 ,'Gauss': 1 ,'T':1e4 , 'mT':1e1 } 

#define proton mass 
MassProt = const.physical_constants["proton mass energy equivalent in MeV"]
#print(MassProt)
MassProt = MassProt[0]* EUnit['MeV']/EUnit[DefEunit]
c_si = const.c* (LUnit['m']/LUnit[DefLunit])
c = 1
print("proton mass : ", MassProt, " ",DefEunit)
print("c : ", c_si , DefLunit+"/s")
print("reltivistic c:",c)
# get cyclotron radious unit a 
def get_a(w0 = 2, c = 1 ):
    return  c/w0   
def get_b(w0 = 2, mass = 0.9382, q = 1 ):
    return (w0*mass)/q
# gamma 
def GetGamma(En = 0.002, mass = 0.9382):
    return (En+mass)/mass
# beta 
def GetBeta(En =0.002, mass = 0.9382):
    return  np.sqrt( 1.0 - 1.0 /np.power(GetGamma(En,mass),2))
# beta gamma 
def BetaGamma(En =0.002, mass = 0.9382):
    return GetBeta(En=En, mass=mass)*GetGamma(En=En, mass=mass)
# get angular velocity 
def GetW(freq = 5e6):
    return 2*np.pi* freq

def GetMom(En= 0.02, a = 299.8/5.063, mass = 0.938 ):
    return BetaGamma(En = En, mass=mass)*a
def GetTotMom(En= 0.002, mass = MassProt, a=1e3, c=c):
    A =(En/(mass*c*c))
    
    return a*np.sqrt(A*(2+A))

def RefField(mass = const.proton_mass, Q = const.elementary_charge, w0 = 2):
    return (mass/Q)*w0

proton mass :  938.27208816   MeV
c :  299792458000.0 mm/s
reltivistic c: 1


In [3]:
En = 0.8685 #MeV 
freq = 5.0630e6  # cyclron freq in si 
CycloHarm = 10
RF = freq*CycloHarm 


print("gamma: \t",   GetGamma(En = 0.002, mass = MassProt))
print("beta: \t",    GetBeta(En = 0.002, mass = MassProt))
print("betagamma: ", BetaGamma(En = 0.002, mass = MassProt) )
print("W0: \t",      GetW(freq=freq))
print("a: \t",       get_a(w0 = GetW(freq=freq) , c =c_si), "["+DefLunit+".Sec/rad]")
print("b: \t",       get_b(w0 = GetW(freq=freq) , mass = MassProt))
print("momentum: ",  GetMom(En= En, a = get_a(w0 = GetW(freq=freq) , c =c_si), mass = MassProt ),
      "[" + DefEunit+ "."+DefLunit+"/a']" )
print("momentum: ",  GetTotMom(En= En, c = c, a  = get_a(w0 = GetW(freq=freq) , c =c_si),  mass = MassProt ),
      "[" + DefEunit+ "."+DefLunit+"/a']" )
print("ref Feild: ", RefField(w0 = GetW(freq=freq)))

gamma: 	 1.0000021315778496
beta: 	 0.002064737772299772
betagamma:  0.0020647421734490724
W0: 	 31811767.210250244
a: 	 9423.948566535537 [mm.Sec/rad]
b: 	 29848093248.42131
momentum:  405.5728685391219 [MeV.mm/a']
momentum:  405.5728685391001 [MeV.mm/a']
ref Feild:  0.33210482625966964


# load field map for transport to occur 

In [5]:
FileName ="inj55a_bfld.dat"
#FileName = "inj72_sector.opal"
#FileName = "ring590_bfld.dat"
#FileName = "ZYKL9Z.dat"

In [6]:
import os
import matplotlib.pyplot as plt 
import numpy as np 
import math

#if its only a partial field map expand it to a full 360 
ExpandToFull =True

#load map 
data_headder = np.genfromtxt(os.path.join(Dir, FileName), max_rows= 6 )
data = np.genfromtxt(os.path.join(Dir, FileName), skip_header= 6 )
#print(data_headder)
#print(data)
if(len(data)>0 and len(data_headder)>0):
    print("File Leaded")
    R0 = data_headder[0]
    dR = data_headder[1]
    Th0= data_headder[2]
    dTh= data_headder[3]
    arcLen = int(data_headder[4])
    n_arcs = int(data_headder[5])
    if(dTh<0):
        symmetry = np.abs(dTh)
        dTh=1/dTh
    print("R0: ", R0, " mm \ndR: ",dR, " mm\nTh0: ",Th0 ,  "deg \ndTh: ",dTh , "deg\narcLen: ",arcLen , "\nn_arcs: ", n_arcs)
    
    data = np.reshape(data,(1, -1))
    data = np.transpose(data)
    #convert field to correct units 
    data = data*(BUnit['kGauss']/BUnit[DefBunit])
    
    #is the map symetric ?
    if(np.mod(int(dTh*arcLen),360) == 0):
        mapsym = 1
        ExpandToFull = False
    else:
        mapsym = np.abs(int(np.floor(360.0/(dTh*arcLen))))
    
    
    #
    if(ExpandToFull):
        fulldata = np.zeros([mapsym*len(data)])
        for i in range(mapsym):
            for j, d in enumerate(data): 
                fulldata[(i*len(data))+j] = d
        
        arcLen*=mapsym
    else:
        fulldata = data
    print("<B> = ", np.mean(fulldata),DefBunit)
    print("B_max = ", np.amax(fulldata),DefBunit)
    print("B_min = ", np.amin(fulldata),DefBunit)
    print("map has symmetry of",mapsym,"Start angle",Th0 , "end angle", dTh*arcLen )     
            
            

File Leaded
R0:  500.0  mm 
dR:  10.0  mm
Th0:  0.0 deg 
dTh:  -0.25 deg
arcLen:  1440 
n_arcs:  294
<B> =  3.0421161496598637 kGauss
B_max =  11.29141 kGauss
B_min =  0.0 kGauss
map has symmetry of 1 Start angle 0.0 end angle -360.0


In [17]:
%run {OpalFunPath}
# some additonals 
print("flutter: ", GetFlutter(dt = fulldata, arclen = arcLen, r = 20, dr = dR))

flutter:  1.946878639254713


# Runge-kutta integrator 
The integrator needed to approximate the particle field influence over a discrete step.

In [18]:
def ode_system(_t, _y):
    """
    system of first order differential equations
    _t: discrete time step value
    _y: state vector [y1, y2]
    """
    return np.array([_y[1], -_t * _y[1] + (2 / _t) * _y[0]])


def rk4(func, tk, _yk, _dt=0.01, **kwargs):
    """
    single-step fourth-order numerical integration (RK4) method
    func: system of first order ODEs
    tk: current time step
    _yk: current state vector [y1, y2, y3, ...]
    _dt: discrete time step size
    **kwargs: additional parameters for ODE system
    returns: y evaluated at time k+1
    """

    # evaluate derivative at several stages within time interval
    f1 = func(tk, _yk, **kwargs)
    f2 = func(tk + _dt / 2, _yk + (f1 * (_dt / 2)), **kwargs)
    f3 = func(tk + _dt / 2, _yk + (f2 * (_dt / 2)), **kwargs)
    f4 = func(tk + _dt, _yk + (f3 * _dt), **kwargs)

    # return an average of the derivative over tk, tk + dt
    return _yk + (_dt / 6) * (f1 + (2 * f2) + (2 * f3) + f4)


# track proton through the field


In [40]:
# dr/dth 
def dr_dth(r=1,Pr =1,Pth= 1):
    try:
        return r*(Pr/Pth)
    except:
        return 0
    
def GetPth(Pr=0, P=0):
    try:
        return np.sqrt(P*P - Pr*Pr)
    except:
        print("error in Pth")
        return 0
        # dPr/dTh = Pth 

def dPr_dTh(Qp =0, r=0, Bz =0, Pr=0, P=0):
    if (r == 0):
        return GetPth(Pr=Pr, P=P)
    else:
        try:
            return Qp*r*Bz
        except: 
            return 1

        
def dx_dTh(Pr = 1, Pth = 1, x = 1, r = 1):
    return 1 


# p = pr, pth , pz ; loc = r, th , z 
def TakeStep(P=[1,1,1], loc=[1,1,1], dTh =1, qPrime = 1, B = 0):
    RKmeth = False
    if(not RKmeth):
        dr = dr_dth(r=loc[0],Pr=P[0],Pth=P[1])*dTh
        dTh = dTh
        dz = 0
        dPr = qPrime* loc[0]*B*dTh
        p = np.sqrt(P[0]**2 + P[1]**2 +P[2]**2 )
        dPth = P[1] - (np.sqrt( p**2 - np.power(P[2]+dPr ,2 )))
        dPz = 0

        
    return [dr,dTh,dz], [dPr,dPth,dPz]
    

In [43]:
# use the dtheta as the step size would be fine but we need to consider time!
# for now dTh

nSteps = np.abs(360.0 /dTh)
En =  0.868 #  MeV kinetic energy 

Pr0 = 0.012
Pz = 0
P = (GetGamma(En = En, mass = MassProt) -1)*MassProt*c
Pth = GetPth(Pr=Pr0, P=P) 

r0  = 340 
th0 = 0 
z0 = 0 

StartPos  = [r0, th0,z0]
StartMom  = [Pr0,Pth,Pz]

print("r0:", r0, "\tTh0:",Th0, "\tZ0:",z0)
print("Ptot:", P, "\tPth0:",Pth, "\tPr0:",Pr0,"\tpZ:",Pz)

for i in range(2):
    #TAKE some steps 
    [dx, p] = TakeStep(P=StartMom, loc=StartPos, dTh = Th0, qPrime = 1, B = 0)
    #implement change 
    
    

r0: 340 	Th0: 0.0 	Z0: 0
Ptot: 0.8680000000000001 	Pth0: 0.8679170467273932 	Pr0: 0.012 	pZ: 0
